In [4]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np


from sklearn.metrics import mean_squared_error
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error

import math

from sklearn.preprocessing import PowerTransformer
import h5py
from keras.models import load_model

ModuleNotFoundError: No module named 'absl'

In [ ]:
df=pd.read_excel("database_new.xlsx")

In [ ]:
df_test=pd.read_csv("../data/inverseproblem_troposdata.csv")

In [ ]:
# GridSearchCV

In [31]:
parameters=[{'alpha':[0.0001, 0.001, 0.01, 0.05, 0.1, 0.5,1],
            'kernel':['rbf'],
            'gamma':[0, 0.0001, 0.001, 0.01,0.05,0.5, 0.75, 1]}]

In [32]:
X = df.iloc[:, [0, 7, 32, 33]]
Y = df.iloc[:, [2]]
X_train, X_test, Y_train, Y_test = train_test_split(
            X, Y,
            test_size=0.5,
            random_state=10)
pt = PowerTransformer(method='box-cox')

X_train_transformed = pt.fit_transform(X_train+0.00000000001)



print(pd.DataFrame({'cols':X_train.columns,'box_cox_lambdas':pt.lambdas_}))

regressor= KernelRidge()
model=regressor.fit(X_train_transformed,Y_train)
grid_search=GridSearchCV(estimator=model,
                        param_grid=parameters,
                         cv=2,
                         scoring='neg_mean_absolute_error',
                        n_jobs=-1
                        )
grid_search= grid_search.fit(X_train_transformed,Y_train)
print('Best score in CV: ', grid_search.best_score_)
print('Best parameters in CV: ', grid_search.best_params_)

parameter_alpha=grid_search.best_params_['alpha']
parameter_kernel=grid_search.best_params_['kernel']
parameter_gamma=grid_search.best_params_['gamma']
print(parameter_alpha)
print(parameter_gamma)

                cols  box_cox_lambdas
0         wavelength        -1.807484
1  equi_mobility_dia         0.195771
2                ssa         0.461682
3          mac_total         1.034593
Best score in CV:  -1.0019024387728832
Best parameters in CV:  {'alpha': 0.0001, 'gamma': 1, 'kernel': 'rbf'}
0.0001
1


In [39]:
X_train = df.iloc[:, [0, 7, 32, 33]]
Y_train = df.iloc[:, [2]]
X_test = df_test.iloc[:, [0, 1,3,2]]
Y_test = df_test.iloc[:, [4]]
pt = PowerTransformer(method='box-cox')

X_train_transformed = pt.fit_transform(X_train+0.00000000001)
X_test_transformed = pt.transform(X_test+0.000000001)

regressor = KernelRidge(alpha=0.0001, gamma=0.01, kernel='rbf')


model = regressor.fit(X_train_transformed,Y_train)


Y_pred_KRR = model.predict(X_test_transformed)
Y_pred_KRR = pd.DataFrame(data=Y_pred_KRR, columns=["fraction_of_coating_pred_KRR"])

error = mean_absolute_error(Y_test, Y_pred_KRR, multioutput='raw_values')

In [40]:
error

array([69.55193277])

In [41]:
X_train = df.iloc[:, [0, 7, 32, 33]]
Y_train = df.iloc[:, [2]]
X_test = df_test.iloc[:, [0, 1,3,2]]
Y_test = df_test.iloc[:, [4]]
pt = PowerTransformer(method='box-cox')

X_train_transformed = pt.fit_transform(X_train+0.00000000001)
X_test_transformed = pt.transform(X_test+0.000000001)




model = load_model('best_model_NN_inverse.hdf5')
print(model)
Y_pred_NN = model.predict(X_test_transformed)
Y_pred_NN = pd.DataFrame(data=Y_pred_NN, columns=["fraction_of_coating_pred_NN"])
#Y_test.reset_index(inplace=True)
#Y_test.drop('index',axis=1, inplace=True)

error = mean_absolute_error(Y_test, Y_pred_NN, multioutput='raw_values')

# error=calculate_mean_absolute_percentage_error_multi(parameter_alpha, parameter_kernel# parameter_gamma, X_train, Y_train, X_test, Y_test, scaling_y)
print('Mean absolute error on test set: ', error)

Mean absolute error on test set:  [36.55528693]


In [43]:
df_all=pd.concat([X_test, Y_test, Y_pred_KRR, Y_pred_NN], axis =1)
df_all

,wavelength,equi_mobility_dia,ssa,mac_total,fraction_of_coating,fraction_of_coating_pred_KRR,fraction_of_coating_pred_NN
0,660,128.9,0.06,4.071666,31.8,11.745920,12.210373
1,660,88.5,0.03,3.755427,31.0,19.554603,20.540613
2,660,56.4,0.00,5.653753,64.4,-28.199178,-1.618268
3,660,111.6,0.14,4.843028,44.6,4.788164,20.796255
4,660,48.3,0.04,12.677165,74.3,-94.410658,-9.703694
5,660,75.0,0.07,3.350569,56.2,34.743777,37.447083
6,660,122.0,0.11,6.072272,33.6,-45.117720,-1.431568
7,660,121.7,0.08,5.922298,37.2,-49.433519,-1.843459
8,660,64.6,0.03,4.855422,23.2,-12.825467,-0.709149
9,660,136.4,0.30,5.222969,35.6,36.354347,47.776928


In [45]:
df_all.to_csv("../data/inverse_prediction_new.csv")